In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer



In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer



In [32]:
df = pd.read_csv('../data/processed/books_dedup.csv')

In [33]:
df.head()

,title,author,rating,rating_count,description,isbn,image,book_url,pages,published_year,language,genres,source
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.35,9953205.0,Winning means fame and fortune. Losing means c...,9780439023481.0,https://m.media-amazon.com/images/S/compressed...,https://www.goodreads.com/book/show/2767052-th...,374.0,2008,English,"Young Adult, Dystopia, Fiction, Fantasy, Scien...",web_scraping
1,Pride and Prejudice,Jane Austen,4.30,4827168.0,"Since its immediate success in 1813, Pride and...",9781441341709.0,https://m.media-amazon.com/images/S/compressed...,https://www.goodreads.com/book/show/1885.Pride...,279.0,1813,English,"Classics, Romance, Fiction, Historical Fiction...",web_scraping
2,To Kill a Mockingbird,Harper Lee,4.26,6911906.0,"""Shoot all the bluejays you want, if you can h...",9780060935467.0,https://m.media-amazon.com/images/S/compressed...,https://www.goodreads.com/book/show/2657.To_Ki...,323.0,1960,English,"Classics, Fiction, Historical Fiction, School,...",web_scraping
3,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.50,3808735.0,It's official: the evil Lord Voldemort has ret...,9780439358064.0,https://m.media-amazon.com/images/S/compressed...,https://www.goodreads.com/book/show/58613451-h...,896.0,2003,English,"Fantasy, Fiction, Young Adult, Harry Potter, M...",web_scraping
4,The Book Thief,Markus Zusak,4.39,2890214.0,Librarian's note: An alternate cover edition c...,NaN,https://m.media-amazon.com/images/S/compressed...,https://www.goodreads.com/book/show/19063.The_...,592.0,2005,English,"Historical Fiction, Fiction, Young Adult, Clas...",web_scraping


In [34]:
df.shape

(1369, 13)

In [35]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1369 entries, 0 to 1368
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1368 non-null   str    
 1   author          1368 non-null   str    
 2   rating          1344 non-null   float64
 3   rating_count    1344 non-null   float64
 4   description     1369 non-null   str    
 5   isbn            1247 non-null   str    
 6   image           1366 non-null   str    
 7   book_url        1369 non-null   str    
 8   pages           1367 non-null   float64
 9   published_year  1369 non-null   int64  
 10  language        1369 non-null   str    
 11  genres          1368 non-null   str    
 12  source          1369 non-null   str    
dtypes: float64(3), int64(1), str(9)
memory usage: 139.2 KB


In [6]:
df[df["title"].isna() | df["author"].isna()][
    ["title", "author", "description", "book_url"]
]

,title,author,description,book_url
205,NaN,NaN,Discover and share books you love on Goodreads.,https://www.goodreads.com/book/show/7190.The_T...


In [7]:
df = df.dropna(subset=["title", "author"])

In [8]:
df.info()

<class 'pandas.DataFrame'>
Index: 1368 entries, 0 to 1368
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1368 non-null   str    
 1   author          1368 non-null   str    
 2   rating          1344 non-null   float64
 3   rating_count    1344 non-null   float64
 4   description     1368 non-null   str    
 5   isbn            1247 non-null   str    
 6   image           1366 non-null   str    
 7   book_url        1368 non-null   str    
 8   pages           1367 non-null   float64
 9   published_year  1368 non-null   int64  
 10  language        1368 non-null   str    
 11  genres          1368 non-null   str    
 12  source          1368 non-null   str    
dtypes: float64(3), int64(1), str(9)
memory usage: 149.6 KB


In [9]:
for col in ["title", "description", "genres"]:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

In [10]:
df["published_year"].describe()

count    1368.000000
mean     1918.690058
std       131.457444
min         0.000000
25%      1893.000000
50%      1952.500000
75%      1996.000000
max      2026.000000
Name: published_year, dtype: float64

In [ ]:
df[df["published_year"] == 0]

In [ ]:
df["published_year"] = df["published_year"].replace(0, np.nan)

In [ ]:
df["pages"].describe()

In [ ]:
#30-40pages/hour 10-15hours 2weeks

In [ ]:
#characters
df["desc_len"] = df["description"].str.len()
df["desc_len"].describe()

In [ ]:
df["desc_word_count"] = df["description"].str.split().str.len()
df["desc_word_count"].describe()

In [ ]:
# too short -> no information.  too long -> noise

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df[["title", "description","genres"]].head()

In [ ]:
# TF-IDF （Term Frequency）-（Inverse Document Frequency）

In [17]:
for col in ["title", "genres", "description"]:
    df[col] = (
        df[col]
        .fillna("")
        .astype(str)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
        .str.lower()
    )

In [18]:
df["tfidf_text"] = (
    (df["title"] + " ") * 3
    + (df["genres"] + " ") * 2
    + df["description"]
)

In [19]:
tfidf = TfidfVectorizer(
    stop_words="english",   # drop the / and / is
    max_features=5000  # unique words   
)

tfidf_matrix = tfidf.fit_transform(df["tfidf_text"])

In [20]:
tfidf_matrix.shape

(1368, 5000)

In [ ]:
#1368 books/rows 5000 unique words/columnes

In [21]:
tfidf.get_feature_names_out()[:20]

array(['000', '01', '02', '07', '10', '100', '11', '12', '13', '15',
       '1564', '16', '1616', '17', '17th', '1800', '1815', '1818', '1847',
       '1850'], dtype=object)

In [22]:
tfidf_matrix[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 62 stored elements and shape (1, 5000)>

In [23]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [24]:
def recommend_books(book_title, df, cosine_sim, top_n=5):
    idx = df[df["title"] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]

    book_indices = [i[0] for i in sim_scores]
    return df.loc[book_indices, ["title", "author"]]

In [25]:
recommend_books(
    book_title=df["title"].iloc[0],
    df=df,
    cosine_sim=cosine_sim
)

,title,author
208,"eat, pray, love",Elizabeth Gilbert
170,"catching fire (the hunger games, #2)",Suzanne Collins
1123,the two towers,J.R.R. Tolkien
252,his dark materials (his dark materials #1-3),Philip Pullman
802,leonardo da vinci,Walter Isaacson


In [29]:
#eg
user_input = "Bedtime Reading"

In [30]:
#TF-IDF
user_input_clean = " ".join(str(user_input).lower().split())
user_vec = tfidf.transform([user_input_clean])

In [31]:
scores = cosine_similarity(user_vec, tfidf_matrix)[0]
top_idx = scores.argsort()[::-1][:5]

result = df.loc[top_idx, ["title", "author"]].copy()
result["score"] = scores[top_idx]
result

,title,author,score
872,saint thomas aquinas,Gilbert Keith Chesterton,0.376875
858,the letters of vita sackville-west to virginia...,Vita Sackville-West,0.228234
529,schachnovelle,Stefan Zweig,0.158440
687,aus meinem leben,Johann Wolfgang von Goethe,0.115252
36,1984,George Orwell,0.109055
